In [51]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder
import os

#1 Carregamento do CSV
nome_arquivo = "dados_tratados.csv"
df = pd.read_csv(nome_arquivo)

In [52]:
#2 Agrupamento por id_da_compra
agrupado = df.groupby("id_da_compra")["produto"].apply(list)

#3 Transações com mais de um produto
transacoes_multiplos = agrupado[agrupado.apply(lambda x: len(set(x)) > 1)]
print(f"\n Transações com múltiplos produtos: {len(transacoes_multiplos)}")


 Transações com múltiplos produtos: 59906


In [53]:
#4 Simula caso não existam transações suficientes
if len(transacoes_multiplos) == 0:
    print(" Simulando transações com múltiplos produtos...")
    df_simulado = df.sample(frac=1).copy()
    df_simulado["id_simulado"] = (df_simulado.index // 5) + 1
    transacoes_multiplos = df_simulado.groupby("id_simulado")["produto"].apply(list)
    print(f"Transações simuladas: {len(transacoes_multiplos)}")

In [54]:
#5 One-hot encoding
te = TransactionEncoder()
te_ary = te.fit(transacoes_multiplos).transform(transacoes_multiplos)
df_binario = pd.DataFrame(te_ary, columns=te.columns_)

In [55]:
#6 FP-Growth
min_support_value = 0.05  # Ajuste o valor conforme necessário
frequent_itemsets = fpgrowth(df_binario, min_support=min_support_value, use_colnames=True)

In [56]:
#7 Sem itens frequentes
if frequent_itemsets.empty:
    print(" Nenhum item frequente encontrado.")
else:
    #8 Regras de associação
    regras = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.2)

    #9 Apenas suporte, confiança e lift, com filtro de qualidade
    regras_qualidade = regras[["antecedents", "consequents", "support", "confidence", "lift"]]
    regras_qualidade = regras_qualidade[
        (regras_qualidade["lift"] > 1) & (regras_qualidade["confidence"] >= 0.2)
    ]
    regras_qualidade = regras_qualidade.sort_values(by="lift", ascending=False)

    #10 Resultado
    print(f"\nRegras de associação encontradas: {len(regras_qualidade)}")
    print(regras_qualidade.head(10))

    #11 Exporta CSV
    regras_qualidade.to_csv("regras_associacao.csv", index=False)
    print("\nArquivo 'regras_associacao.csv' salvo com sucesso!")


Regras de associação encontradas: 53
          antecedents        consequents   support  confidence      lift
59          (Cerveja)     (Refrigerante)  0.065720    0.585080  4.619714
58     (Refrigerante)          (Cerveja)  0.065720    0.518914  4.619714
61          (Cerveja)  (Suco De Laranja)  0.062615    0.557438  4.583294
60  (Suco De Laranja)          (Cerveja)  0.062615    0.514823  4.583294
21  (Suco De Laranja)     (Refrigerante)  0.069559    0.571919  4.515799
22     (Refrigerante)  (Suco De Laranja)  0.069559    0.549229  4.515799
36  (Molho De Tomate)            (Arroz)  0.062665    0.369779  4.329946
35            (Arroz)  (Molho De Tomate)  0.062665    0.733776  4.329946
34           (Feijão)  (Molho De Tomate)  0.053734    0.540736  3.190831
33  (Molho De Tomate)           (Feijão)  0.053734    0.317080  3.190831

Arquivo 'regras_associacao.csv' salvo com sucesso!


In [57]:
#12 Ranking dos produtos mais vendidos
if 'produto' in df.columns:
    ranking_produtos = df['produto'].value_counts().reset_index()
    ranking_produtos.columns = ['produto', 'quantidade_vendida']

    print("\n Top 10 Produtos Mais Vendidos:")
    print(ranking_produtos.head(10))

    ranking_produtos.to_csv("ranking_produtos.csv", index=False)
    print("Arquivo 'ranking_produtos.csv' salvo com sucesso!")


 Top 10 Produtos Mais Vendidos:
            produto  quantidade_vendida
0    Pasta De Dente               25824
1  Queijo Mussarela               25762
2          Sabonete               24411
3          Manteiga               23868
4              Café               22013
5            Açúcar               21787
6      Papel Toalha               18433
7      Desinfetante               18252
8   Molho De Tomate               12349
9     Condicionador               11249
Arquivo 'ranking_produtos.csv' salvo com sucesso!


In [58]:
# #13 Ranking de estados com mais vendas
if 'estado' in df.columns:
    ranking_estados = df['estado'].value_counts().reset_index()
    ranking_estados.columns = ['estado', 'quantidade_vendas']

    print("\n Top Estados com Mais Compras:")
    print(ranking_estados.head(10))

    ranking_estados.to_csv("ranking_estados.csv", index=False)
    print(" Arquivo 'ranking_estados.csv' salvo com sucesso!")


 Top Estados com Mais Compras:
  estado  quantidade_vendas
0     MG             225726
1     RJ              98044
2     SP              38728
3     SC                147
4     PR                140
5     RS                135
6     MT                130
 Arquivo 'ranking_estados.csv' salvo com sucesso!
